Try nmf with all non negative features with grid search of c 


# NMF Modeling for player recomendations

non negative matrix factorization(NMF) is a feature reduction technique that decomposes a matrix into two smaller matricies that can be used to cluster features from the original matrix into topics as well as calculate the distance between samples in the original matrix to allow for recomendations between the closest samples to be created. I will be using NMF to generate player recomendations to find who has the cloest stats to  'Walker Zimmerman' and some of the other best players in the MLS 

In [1]:
## import packages and tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [2]:
## load data
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [3]:
## set index and drop negative and categotical columns 
X = df.set_index('player_name')

X = X.drop(columns= ['Base Salary','Player', 'Club',
                     'Nation', 'Position','fb_id', 'Squad_from_op','G-xG','np:G-xG'])
X.shape

(705, 100)

In [4]:
## select key features to input into NMF 
X = X[[ 'Tackle_pct_won', 'Nineties', 'xA',
       'Key_pass', 'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct', 'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
       'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
       'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
       'PK_won', 'Aerial_win_pct', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
       'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG',
       'npxG/Sh', 'pAdj_Total_tackles', 'pAdj_Tackles_Won',
       'pAdj_Tackles_Def_3rd', 'pAdj_Tackles_Mid_3rd', 'pAdj_Tackles_Att_3rd',
       'pAdj_Num_Dribblers_tackled', 'pAdj_Num_Dribbled_past', 
       'pAdj_Blocked_shots', 'pAdj_Blocked_SOT', 'pAdj_Blocked_pass',
       'pAdj_Interceptions', 'pAdj_Tackles_and_Ints', 'pAdj_Clearences',
       'pAdj_Yellow_cards', 'pAdj_Red_cards',
       'pAdj_Fouls', 'pAdj_def_interceptions', 
       'pAdj_PK_conceded', 'pAdj_OG', 'pAdj_Recoveries',
       'pAdj_Aerial_Duels_lost', 'lost_tackles', 'True_tackle_pct',
       'pAdj_Tackle_int_blocks', 'avg_shrt+med_pass_pct', 'pct_long_balls',
       'prog_carry+lng_comp+crosses', 'attacking_touches']]
X = X.loc[(X['Nineties']>5)]
X.shape

(541, 90)

# Overview -

NMF or Non Negative Matrix Factorization is widley used for topic modeling and document clustering but there are many examples of NMF being used to cluster other kinds of datasets such as classifying Companies on extra finacial criteria.https://towardsdatascience.com/using-nmf-to-classify-companies-a77e176f276f .
    I am going  to use NMF to classify my MLS 2021 data set by the "topic" or ,grouping of player stats,that I will call "skill set group". I will use the skill sets to classify players in the league and then use a normalized version of the decomposed matrix to identify players who are closest to Key players Identified in my EDA such and "Nathan" and " "Zimmerman". At the end I should have all players classified by a key skill set and players closest to the target players identifed so show how francises could use NMF to I dentify players from around the globe that could fit into roles played by players in the MLS with wider data sets than I have access to at this time. 

# NMF matrix break down -
With the player matrix X being of n players(rows) and f features(player stats) (n x f)
and 
c = number of Skill set groups generated 

## Weight matrix - weights of each row for each skill set group or topic 
 players times skil set matrix 
#W = n x c 

## Hidden layer matric - hidden layer of feature value for each skill set group
player stat features times skill set group matrix
H = F x c

## Thus the matric multiplcation of W (weight)* H(hidden) approximates the original X player matrix

 W x H approximates X

## The W weight matrix is than used to find players closest to the target player in question

The distane between the topic values fo each player in the W matrix can be used to measure who is the most simular allowing for recomendations of players closest to the target player to be created.

The skill set group that has the higest ranking for each player is used to classify or label groups of players who are simular in their player stats features. 



In [11]:
# function to viz 
def display_features(H,W,feature_names, X_matrix ,no_top_features, no_top_players):
    """ visualize skill get group and highest ranked players in group """
    
    # iterate through topics in topic-term matrix, 'H' aka
    # H is the hidden layer which is shape (F x C) feature times topic matrix
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_features - 1:-1]]))
        top_player_indicies = np.argsort( W[:,topic_idx] )[::-1][0:no_top_players]
        for p_index in top_player_indicies:
            
            print(p_index," ",X_matrix.index[p_index])


In [14]:

c = 11
# Create a MaxAbsScaler: scaler
transformer = MaxAbsScaler().fit(X)

# scale data
scaled_X= transformer.transform(X)

# Create an NMF model: nmf

nmf = NMF(n_components=c,max_iter=100000,init='nndsvda',alpha=.1, l1_ratio=.5, random_state=42)

W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)

            
feature_names = X.columns.values
no_top_features = 4
no_top_players = 5

display_features(H,W,feature_names, X, no_top_features,no_top_players)

NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=11,
    random_state=42) error : 15.986014641229442 
  W shape (541, 11) H shape  (11, 90)
Topic 0:
recept_pct (1.71) Short_Cmp_pct (1.67) avg_shrt+med_pass_pct (1.66) Medium_Cmp_pct (1.65)
488   Brad Guzan
518   Maxime Crépeau
526   Zac MacMath
530   David Ochoa
428   J.T. Marcinkowski
Topic 1:
touches_att_box (1.8) SoT (1.55) npxG (1.54) pAdj_Yellow_cards (1.51)
45   Valentín Castellanos
192   Adam Buksa
106   Ola Kamara
147   Cristian Arango
130   Patryk Klimala
Topic 2:
SoT% (2.29) G/SoT (1.46) npxG/Sh (0.99) G/Sh (0.75)
459   Ralph Priso-Mbongue
283   Paxten Aaronson
505   Jalil Anibaba
463   Tsubasa Endoh
331   Miguel Berry
Topic 3:
touches_mid_3rd (1.43) recep_succ (1.42) carries (1.3) Total_Cmp (1.29)
313   Andreu Fontàs
0   Carles Gil
95   Andrés Ricaurte
108   Haris Medunjanin
323   Ilie Sánchez
Topic 4:
Short_Cmp (1.42) Short_Att (1.39) touches (1.08) Total_Att (1.07)
469   Oniel Fisher
306   D.J. Tay

In [15]:
def add_topics(X,W):
    df_new = X.copy()
    # Get the top predicted topic
    predicted_topics = [np.argsort(each)[::-1][0] for each in W]
    # Add to the df
    df_new['pred_topic_num'] = predicted_topics
    return df_new


df_new = add_topics(X,W)
df_new['pred_topic_num'].describe()

count    541.000000
mean       5.053604
std        3.256175
min        0.000000
25%        3.000000
50%        5.000000
75%        9.000000
max       10.000000
Name: pred_topic_num, dtype: float64

In [16]:
df_new['pred_topic_num'].value_counts()

9     106
4      84
1      78
3      56
6      50
0      44
10     42
5      41
8      18
7      17
2       5
Name: pred_topic_num, dtype: int64

In [17]:
df_new[df_new['pred_topic_num']==2]

,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,Total_Tot_Dist,Total_Prg_Dist,...,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches,pred_topic_num
player_name,,,,,,,,,,,,,,,,,,,,,
Paxten Aaronson,33.3,6.6,0.196970,1.515152,2.575758,21.666667,29.393939,73.7,323.484848,93.181818,...,12.776700,1.252618,2.254712,43.902439,6.513612,77.7,6.701031,7.121212,31.818182,2
Indiana Vassilev,23.3,8.9,0.044944,0.674157,2.022472,23.258427,27.640449,84.1,378.764045,67.640449,...,7.424593,0.543263,0.362175,13.793103,1.991964,89.3,11.788618,6.404494,22.584270,2
Eddie Munjoma,25.7,5.2,0.076923,0.961538,3.653846,37.500000,48.461538,77.4,697.884615,206.153846,...,11.664252,2.268049,1.620035,38.709677,7.128154,84.1,19.047619,11.730769,19.423077,2
Ralph Priso-Mbongue,32.3,6.3,0.015873,0.317460,3.809524,42.380952,50.793651,83.4,726.031746,174.920635,...,11.981049,2.549159,1.784412,34.482759,10.706470,89.2,13.437500,7.777778,14.444444,2
Cameron Duke,24.5,7.4,0.081081,0.270270,0.540541,30.135135,34.054054,88.5,529.594595,63.783784,...,12.392545,1.402930,1.402930,32.500000,6.547005,90.6,11.507937,8.108108,24.189189,2


In [18]:
transformer = Normalizer().fit(W)
normed_X= transformer.transform(W)
# can add back club and salary here 
normed_df = pd.DataFrame(normed_X,index=X.index.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 541 entries, Carles Gil to Oriol Rosell
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       541 non-null    float64
 1   1       541 non-null    float64
 2   2       541 non-null    float64
 3   3       541 non-null    float64
 4   4       541 non-null    float64
 5   5       541 non-null    float64
 6   6       541 non-null    float64
 7   7       541 non-null    float64
 8   8       541 non-null    float64
 9   9       541 non-null    float64
 10  10      541 non-null    float64
dtypes: float64(11)
memory usage: 50.7+ KB


In [23]:
jm = normed_df.loc['Albert Rusnák']
sims = normed_df.dot(jm)

sims.nlargest(10)

Albert Rusnák           1.000000
Diego Fagúndez          0.925886
Javier Eduardo López    0.922795
Yimmi Chará             0.916332
Djordje Mihailovic      0.914978
Cole Bassett            0.906067
Nani                    0.896653
Randall Leal            0.896131
Hany Mukhtar            0.890344
Maximiliano Moralez     0.881918
dtype: float64

Having another creative attacking midfielder in Diego Fagundez as the closest player to Abert Rusnak helps validats the skill set groups created. 

# looking for coherent groups of features inside of components to pick c  

In [20]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson     1.000000
Julian Araujo      0.952665
Zorhan Bassong     0.945047
Justin Che         0.944947
Donny Toia         0.942724
Marco Farfan       0.942453
Oniel Fisher       0.937562
D.J. Taylor        0.937515
Niko Hämäläinen    0.934689
Emmanuel Más       0.929926
dtype: float64

In [21]:
wz = normed_df.loc['Walker Zimmerman']
sims = normed_df.dot(wz)

sims.nlargest(10)

Walker Zimmerman     1.000000
Bill Tuiloma         0.978428
Tanner Beason        0.977045
Jonathan Mensah      0.974217
Ranko Veselinović    0.972588
Jack Maher           0.958475
Robin Jansson        0.955465
Nkosi Burgess        0.954752
Andy Rose            0.954397
Antonio Carlos       0.953612
dtype: float64

In [40]:
jm = normed_df.loc['Jonathan Mensah']
sims = normed_df.dot(jm)

sims.nlargest(10)

Jonathan Mensah      1.000000
Antonio Carlos       0.996214
Rudy Camacho         0.994820
Andy Rose            0.993489
Larrys Mabiala       0.993116
Julio Cascante       0.993039
Josh Williams        0.989379
Nkosi Burgess        0.988423
Matt Hedges          0.986246
Ranko Veselinović    0.984544
dtype: float64

In [22]:
jm = normed_df.loc['Nathan Cardoso']
sims = normed_df.dot(jm)

sims.nlargest(10)

Nathan Cardoso          1.000000
Andrés Reyes            0.960560
Donovan Pines           0.930676
Alan Franco             0.924164
Johan Kappelhof         0.916571
Rodrigo Schlegel        0.912953
Yeimar Gómez Andrade    0.909612
Amro Tarek              0.898251
Nick Hagglund           0.894553
Mamadou Fall            0.893209
dtype: float64